#  JUMIA WEB SCRAPPER

In [935]:
# used libraries
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date
import os
import time

In [936]:
#recording start time
start=time.time()

In [937]:
#opening web driver 
driver = webdriver.Chrome( 'C:\\Users\\youssef\\Desktop\\chromedriver_win32\\chromedriver.exe' ) # insert your web driver path here 

In [938]:
today=date.today()
d1 = today.strftime("%d/%m/%Y")

###                                     FUNCTIONS 

###  item informations exctraction fonction

In [939]:
def extract(item,a):
    #infos
    name = item.find('h3','name').text
    
    brandroot = item.find('a','core')
    brand = brandroot.get('data-brand')
    try:
          discount = item.find('div','tag _dsct _sm').text
    except AttributeError:
        discount = '0'
        
    
    price = item.find('div','prc').text 
    
    try:
        old_price = item.find('div','old').text
    except AttributeError:
           old_price = 'none' 
        
        
    category = brandroot.get('data-category')
    
    try:
        review=item.find('div','stars _s').text
    except AttributeError:
        review ='absent'
    
    if item.get('class') == ['prd', '_fb', 'col', 'c-prd'] : spnsrd =0
    else : spnsrd =1
     
    try:
        picture=item.find('img',{'alt':''})
        picture_url=picture.get('src')
        
    except AttributeError:
        picture_url='none'
        
        
    extracted=(name,brand,category,price,old_price,discount,review,spnsrd,picture_url ,d1,a)
    
    return(extracted)
    

### categorys url creation fonction:

In [941]:
def urlmaker(href):
        if href.find('jumia.ma')  == -1 :
            if href.find('?shipped_from=country_local') != -1:
                    A=['https://www.jumia.ma/',href,'&page=0#catalog-listing']
                    separator = ''
                    link=(separator.join(A))
                    
    
            else :
                     A=['https://www.jumia.ma/',href,'?page=0#catalog-listing']
                     separator = ''
                     link=(separator.join(A))
           
            return(link)

### max pages extraction fonction 

In [942]:
def numbrpg(url):
    driver.get(url)
    try:
        soups= BeautifulSoup(driver.page_source, 'html.parser')
        a= soups.find('div',{'class':'pg-w -ptm -pbxl'})
        b=a.find('a',{'aria-label':'Dernière page'})
        c=b.get('href')
        start=c.find('page=')+5
        end=c.find('#catalog-listing')
        i=int(c[start:end])
    except AttributeError :
        i=1
    return(i)

## links extractions :

In [926]:
driver = webdriver.Chrome( 'C:\\Users\\youssef\\Desktop\\chromedriver_win32\\chromedriver.exe' )

In [943]:
url='https://www.jumia.ma/epicerie-boissons/?shipped_from=country_local'
driver.get(url)

In [949]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [950]:
link_id=[]
link_id=  soup.find_all('div',{'class' : 'sub'},{'role','menu'})

In [951]:
links=[]
for j in link_id :
    test=j.find_all('a',{'role':'menuitem'})
    for k in test :
            A=[]
            href=str(k.get('href'))
            if href !='None' :
                    link=urlmaker(href)
                    if link != None :
                        A.append([])
                        A.append(link)
                        A.append(k.text)
                        A.append(numbrpg(link))
                        links.append(A)
               

In [ ]:
links

In [842]:
driver.close()

In [953]:
print(len(links))
for j in links :
    if j[1]==None:
        print(j)

341


In [954]:
final_links=[]
for j in links :
     if j[1]!=None:
            final_links.append(j)

In [955]:
len(final_links)

341

In [956]:
links[0]

[[],
 'https://www.jumia.ma//epicerie-boissons/?shipped_from=country_local&page=0#catalog-listing',
 'Eaux, boissons',
 24]

## items data extraction

In [826]:
driver = webdriver.Chrome( 'C:\\Users\\youssef\\Desktop\\chromedriver_win32\\chromedriver.exe' )

In [957]:
for j in links : 
    
        for i in range (j[3]):
            url=j[1]
            urls = url.replace('0',str(i+1))
            
            driver.get(urls)
           
            soup = BeautifulSoup(driver.page_source, 'html.parser')
    
            mainlist = soup.find_all('article',{'class' : 'prd _fb col c-prd'})
            sponsord = soup.find_all('article',{'class': 'prd _fb _spn c-prd col'})
    
            for item in mainlist:
                j[0].append(extract(item,j[2]))
            
            for item in  sponsord :
                j[0].append(extract(item,j[2]))                         
    

In [ ]:
x=0
for j in links:
    print(j[2],len(j[0]))
    x=x+len(j[0])


In [959]:
x

395163

In [960]:
for j in links :
    
        cld=[]
        cld = list(dict.fromkeys(j[0]))
        j[0]=cld


In [ ]:
x=0
for j in links:
    print(len(j[0]))
    x=x+len(j[0])


## joining data

In [962]:
final=[]
for j in links :
    final=final+j[0]

In [963]:
cld=[]
cld = list(dict.fromkeys(final))
final=cld

In [964]:
a_set=set(final)
contains_duplicates = len(final) != len(a_set)
print(contains_duplicates)

False


## saving data

In [965]:
A=['data of',str(today)]
separator = ' '
dirname=(separator.join(A))

In [966]:
dir = os.path.join("C:\\Users\\youssef\\Desktop\\WEBScrapper python\\data",dirname)#saving directory here
if not os.path.exists(dir):
    os.mkdir(dir)
os.chdir(dir)
cwd = os.getcwd()
print(cwd)

C:\Users\youssef\Desktop\WEBScrapper python\data\data of 2021-08-01


In [967]:
with open('results_final.csv','w',newline='',encoding='UTF-8') as f:
    writer=csv.writer(f)
    writer.writerow(['name','brand','category','price','old price','discount','review','sponsorship','picture url','date','site categoration'])
    writer.writerows(final)

In [ ]:
#to export data as individual sets 
for j in final_links :
    
    a=[j[2],'data of',str(today),'.csv']
    separator = ' '
    dataname=(separator.join(a))
    with open(dataname,'w',newline='',encoding='UTF-8') as f:
        writer=csv.writer(f)
        writer.writerow(['name','brand','category','price','old price','discount','review','sponsorship','picture url','date','site categoration'])
        writer.writerows(j[0])

        

In [970]:
dir = os.path.join("C:\\Users\\youssef\\Desktop")
os.chdir(dir)

## process duration 

In [969]:
end=time.time()
print(end - start)

13649.464428186417
